## Network Failures - Oracle

- **Goal**: to understand the impact of an impedence (deliberately or accidental) of the oracle price signal of USD/POKT on reward incentives
- **System Goals Targeted**:
  - Economic Viability
  - Long-Term Value Generation
- **Design**: allow a 'true' oracle signal to be impeded without the knowledge of network participants
- **Testing mechanism**:
  - The true oracle signal is impeded by e.g.
    -  Interruption
    -  Delay
    -  Distortion (signal-to-noise ratio is reduced)
  - The impact on 1) servicer and gateway profitability, and 2) token issuance is assessed under different block reward and fee structures
- **KPIs**:
  - Servicer NPV (KPI-1)
  - Gateway NPV (KPI-3)
  - POKT Circulating Supply (KPI-4)
  - POKT Available Supply (KPI-5)
  - Mint/burn rate (KPI-D)
  - DAO Value Capture (KPI-10)
  - Servicer Capital Costs KPI-14)

## Protocol Parameters to be swept

1. `RelaysToTokensMultiplier` (`RTTM`)
2. `GatewayFeePerRelay` (`GFPR`)
3. `ApplicationFeePerRelay` (`AFPR`, assumed different from `GFPR`)
4. `GatewayMinimumStake` (`GMS`)
5. `ApplicationMinimumStake` (`AMS`)
6. `DAOAllocation` (`DAL`)
7. `ValidatorAllocation` (`VAL`) 
8. `ServicerAllocation` (derived parameter, defined as 1 - `DAOAllocation` - `ValidatorAllocation`)

## Exogenous Parameters to be swept
1. Mean of the lognormal distribution used to model oracle price process: $\mu \in (-\infty, \infty)$
2. Standard deviation of the lognormal distribution used to model the oracle price process: $\sigma > 0$
3. Mean of the exponential distribution used to model the interarrival time of the oracle price process: $m > 0$

## Exogenous Distributions that are fixed for this scenario (but may depend upon a parameter being swept)

1. The oracle price process, $p_t \sim e^{\mu + \sigma X}$, with $X \sim \mathcal N(0, 1)$, with dependence upon the sweep parameters $\mu$ and $\sigma$ (cf. above);
2. The interarrival time of oracle prices, $\delta t \sim Exp(m)$, with dependence upon the sweep parameter $m$ as the **scale** (i.e. inverse rate). Cf. e.g. `numpy.random.exponential(m)` (https://numpy.org/doc/stable/reference/random/generated/numpy.random.exponential.html)

## Simulation Initialization

1. Initialize a set $N_s$ of servicers
2. Initialize a set $N_c$ of services (chains)
3. Initialize a set $N_a$ of applications
4. Initialize a set $N_g$ of gateways
5. Initialize the network of applications - gateways - services - servicers
6. Initialize the number of timesteps $T$ even
7. Initialize default values $(\mu_0, \sigma_0)$ for the oracle price process
8. Initialize the interarrival time between oracle prices to be _exactly_ 1, i.e. initially the oracle provides precisely one observation per timestep (day)



## Simulation A - oracle interruption
Simulation runs with default behavior for $T/2$ timesteps, to 'burn in' ecosystem profit/loss to servicers from services

In timestep $T/2 + 1$:
1. The oracle price process is permanently turned off--no new prices are provided to the ecosystem

**Simulation then runs for the remaining $T/2$ timesteps, with the network using the last provided price, $p(T/2)$, as the price of USD/POKT.**



## Simulation B - oracle delay
Simulation runs with default behavior for $T/2$ timesteps, to 'burn in' ecosystem profit/loss to servicers from services

In timestep $T/2 + 1$:
1. The sweep parameter value for $m$ is used to generate an interarrival time realization $\Delta t$ from the exponential distribution (floored to the nearest integer)--this corresponds to the number of timesteps the oracle price is fixed at its value in timestep $T/2$

**Simulation then runs for $T/2 + \lfloor \Delta t \rfloor$ timesteps, with the network using the last provided price, $p(T/2)$, as the price of USD/POKT.**

Note that $\lfloor \Delta t \rfloor$ may be greater than $T - T/2$--in that case, the oracle distortion is permanent, and the following may never occur (in which case the simulation ends in the state above).

In timestep $T/2 + \lfloor \Delta t \rfloor +1$:
1. The interarrival time is returned to a fixed value of _exactly_ 1, i.e. one oracle price is provided every timestep


**Simulation then runs for the remaining $T - T/2 - \lfloor \Delta t \rfloor$ timesteps, with the network operating under the same conditions as the first $T/2$ timesteps.**


## Simulation c - oracle distortion
Simulation runs with default behavior for $T/2$ timesteps, to 'burn in' ecosystem profit/loss to servicers from services

In timestep $T/2 + 1$:
1. The sweep parameter values $(\mu, \sigma)$ are used for the oracle price distribution, reflecting a departure from the 'true' values $(\mu_0, \sigma_0)$;
2. The sweep parameter value for $m$ is used to generate an interarrival time realization $\Delta t$ from the exponential distribution (floored to the nearest integer)--this corresponds to the number of timesteps the oracle price distortion is in effect.

**Simulation then runs for $T/2 + \lfloor \Delta t \rfloor$ timesteps, with the network using the distorted oracle price process to receive the price of USD/POKT.**

Note that $\lfloor \Delta t \rfloor$ may be greater than $T - T/2$--in that case, the oracle distortion is permanent, and the following may never occur (in which case the simulation ends in the state above).

In timestep $T/2 + \lfloor \Delta t \rfloor +1$:
1. The oracle price distribution parameters are returned to $(\mu_0, \sigma_0)$;
2. The interarrival time is returned to a fixed value of _exactly_ 1, i.e. one oracle price is provided every timestep


**Simulation then runs for the remaining $T - T/2 - \lfloor \Delta t \rfloor$ timesteps, with the network operating under the same conditions as the first $T/2$ timesteps.**


## Full Grid Parameter Sweep Values (10 parameters in total)

|  Name | Sweep Values | Units |
| --- | ---| ---|
| `RTTM` | (100, 150, 200) | uPOKT/relay |
| `GFPR` | (20, 25, 30) | uPOKT/relay |
| `AFPR` | (20, 25, 30) | uPOKT/relay |
| `GMS`  | (1e5, 1.5e5, 2e5) | POKT ||
| `AMS`  | (1e4, 1.5e4, 2e4)  | POKT |
| `DAL`  | (0.05, 0.10, 0.15) | % |
| `VAL`  | (0.01, 0.05, 0.10) | % |
| $\mu$  | (-4, -10, -20) | number |
| $\sigma$  | (1.0258966083412198, 3.612819377024821, 5.74912722515392) | number |
| $m$  | (1, 0.10, 0.01) | number |

**Total number of parameter constellations**:
- Routine $A$: excludes $\mu$, $\sigma$, $m$ sweeps: $3^7 = 2,187$
- Routine $B$: excludes $\mu$, $\sigma$ sweeps: $3^8 = 6,561$
- Routine $C$: $3^10 = 59,049$

**Total number of Monte Carlo runs per constellation**: ? 30

**Total number of experiments**: 2.034 million

## Adaptive Grid Parameter Sweep Values (10 parameters in total)

|  Name | Sweep Values | Units |
| --- | ---| ---|
| `RTTM` | (100, 200) | uPOKT/relay |
| `GFPR` | (20, 30) | uPOKT/relay |
| `AFPR` | (20, 30) | uPOKT/relay |
| `GMS`  | (1e5, 2e5) | POKT ||
| `AMS`  | (1e4, 2e4)  | POKT |
| `DAL`  | (0.05, 0.15) | % |
| `VAL`  | (0.01, 0.10) | % |
| $\mu$  | (-4, -10, -20) | number |
| $\sigma$  | (1.0258966083412198, 3.612819377024821, 5.74912722515392) | number |
| $m$  | (1, 0.10, 0.01) | number |

**Total number of parameter constellations**:
- Routine $A$: excludes $\mu$, $\sigma$, $m$ sweeps: $2^7 = 128$
- Routine $B$: excludes $\mu$, $\sigma$ sweeps: $2^7 \times 3 = 384$
- Routine $C$: $2^7 \times 3^3 = 3,456$

**Total number of Monte Carlo runs per constellation**: ? 30

**Total number of experiments**: 119,040

## Loop algorithm (as many times as is feasible given resource/time constraints)

1. Run experiments
2. Assess KPIs via threshold inequalities/sensitivity analysis
3. Select representative 'best' constellation
4. Generate new $(min,max)$ grid sweep values for each parameter around 'best' constellation values
5. Loop--back to #1.

*Note that in the above, the ranges for $\mu$ and $\sigma$ are based upon the surface given for a realistic oracle price (cf. https://docs.google.com/spreadsheets/d/1QYe6NzuiyimsXs5cT1BSM-UT1DtX_K38cOZsEFJOtdA/edit?usp=sharing):
$$\ln(0.031) \simeq -3.473768074496991 = \mu + \frac{1}{2}\sigma^2.$$
_Pairwise_ (i.e. pairing up $\mu_1$ with $\sigma_1$, etc. from the resp. vectors of sweep values given) the sweep values lie on this surface--the _cross-pair_ realizations that occur from the simulation sweeps thus provide more extreme 'out of sample' alternative valuations of POKT in terms of USD. For example, pairing $\mu_1 = -4$ with $\sigma_3 = 5.74\ldots$ yields an expected oracle price of c. USD $275469$, while pairing $\mu_3 = -20$ with $\sigma_1 = 1.02\ldots$ gives an expectation c. USD $3e-9$.

Thus, the 'canonical' pairings $(\mu_i, \sigma_i)$, $i = 1,2,3$ of the sweep values always provide measures of different scales/variances to the price process ("distortions"), _but the same expected value of the oracle price_, $\bar p = 0.031$ USD/POKT.